### **Module 1.2: Manual Logging with MLflow** 


## 🎯 **Learning Objectives Expanded**

### 1️⃣ **Manually Log Hyperparameters (`alpha`)**

* **What it means:**
  Explicitly tracking the values of hyperparameters you use when training models, such as the regularization strength (`alpha`), directly within your code.

* **Detailed Steps:**

  * Specify hyperparameter explicitly:

    ```python
    alpha = 0.1
    ```
  * Log the hyperparameter in MLflow:

    ```python
    import mlflow
    mlflow.log_param("alpha", alpha)
    ```

* **Why it matters:**
  Tracking hyperparameters makes it easy to understand which settings lead to the best performance, improving reproducibility and experiment tracking.

---

### 2️⃣ **Log Multiple Metrics (`mse`, `rmse`)**

* **What it means:**
  Recording different evaluation metrics to assess model performance, such as Mean Squared Error (MSE) and Root Mean Squared Error (RMSE).

* **Detailed Steps:**

  * Calculate your metrics after prediction:

    ```python
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    ```
  * Log metrics explicitly:

    ```python
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    ```

* **Why it matters:**
  Logging multiple metrics provides a comprehensive view of model quality, allowing for more informed decisions when comparing models.

---

### 3️⃣ **Iterate Over Multiple Model Configurations**

* **What it means:**
  Automatically training and logging multiple models with different configurations (hyperparameters) using loops.

* **Detailed Steps:**

  ```python
  alphas = [0.01, 0.1, 1.0, 10.0]
  for alpha in alphas:
      with mlflow.start_run():
          mlflow.log_param("alpha", alpha)
          
          model = Ridge(alpha=alpha)
          model.fit(X_train, y_train)
          
          y_pred = model.predict(X_test)
          mse = mean_squared_error(y_test, y_pred)
          rmse = mse ** 0.5
          
          mlflow.log_metric("mse", mse)
          mlflow.log_metric("rmse", rmse)
  ```

* **Why it matters:**
  Automating multiple runs efficiently helps identify optimal configurations, saving time and effort during model tuning.

---

### 4️⃣ **Compare and Sort Runs Using `mlflow.search_runs()`**

* **What it means:**
  Programmatically retrieving and analyzing logged model runs to identify top-performing models based on metrics.

* **Detailed Steps:**

  ```python
  runs_df = mlflow.search_runs(experiment_names=["manual-logging-ridge"])
  runs_df_sorted = runs_df.sort_values(by="metrics.rmse")
  print(runs_df_sorted[["run_id", "params.alpha", "metrics.mse", "metrics.rmse"]])
  ```

* **Why it matters:**
  Quickly sorting and analyzing experiments allows you to identify which model configurations yield the best performance.


In [1]:
# 📓 Module 1.2: Manual Logging with MLflow
# Goal: Learn how to log multiple runs with varying parameters manually and compare results in the MLflow UI or programmatically.

# ✅ Step 1: Install necessary packages
!pip install -q mlflow scikit-learn

# ✅ Step 2: Import libraries
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# ✅ Step 3: Load dataset and prepare train/test split
# The diabetes dataset is a standard regression dataset available in sklearn
# We will use it to predict disease progression using patient features
diabetes = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2, random_state=42
)

# ✅ Step 4: Set up experiment
# All runs will be grouped under this named experiment for easy comparison
mlflow.set_experiment("manual-logging-ridge")

# ✅ Step 5: Run experiments manually with different alpha values
# We will train a Ridge regression model using various values of alpha (regularization strength)
alphas = [0.01, 0.1, 1.0, 10.0]

for alpha in alphas:
    # Start a new MLflow run
    with mlflow.start_run():
        # Log the hyperparameter 'alpha'
        mlflow.log_param("alpha", alpha)

        # Train Ridge regression model
        model = Ridge(alpha=alpha)
        model.fit(X_train, y_train)

        # Predict on the test set and evaluate performance
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        # Log evaluation metrics
        mlflow.log_metric("mse", mse)     # Mean Squared Error
        mlflow.log_metric("rmse", rmse)   # Root Mean Squared Error

        # Log the trained model as an artifact
        mlflow.sklearn.log_model(model, "ridge_model")

        # Print run info to confirm logging
        run_id = mlflow.active_run().info.run_id
        print(f"Logged run with alpha={alpha}, Run ID={run_id}")

# ✅ Step 6: Compare runs
# Fetch all completed runs in the experiment and sort by RMSE to find best performing configuration
runs_df = mlflow.search_runs(experiment_names=["manual-logging-ridge"])
display_cols = ["run_id", "params.alpha", "metrics.mse", "metrics.rmse"]
runs_df[display_cols].sort_values("metrics.rmse")


2025/08/02 21:39:40 INFO mlflow.tracking.fluent: Experiment with name 'manual-logging-ridge' does not exist. Creating a new experiment.
2025/08/02 21:39:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/02 21:39:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/02 21:39:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logged run with alpha=0.01, Run ID=3b9ab8905ae044dda149d262150627cd


2025/08/02 21:39:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/02 21:39:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logged run with alpha=0.1, Run ID=268c7a65e7564d179eefa431a52f2fe1


2025/08/02 21:39:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/02 21:39:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logged run with alpha=1.0, Run ID=76a08172a4d8470bb22437c7ba10bac3


2025/08/02 21:39:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged run with alpha=10.0, Run ID=aa1b45c42c364478a8841d6f750802bb


,run_id,params.alpha,metrics.mse,metrics.rmse
2,268c7a65e7564d179eefa431a52f2fe1,0.1,2856.486888,53.446112
3,3b9ab8905ae044dda149d262150627cd,0.01,2882.290180,53.686965
1,76a08172a4d8470bb22437c7ba10bac3,1.0,3077.415939,55.474462
0,aa1b45c42c364478a8841d6f750802bb,10.0,4443.952637,66.662978


## 📝 Assessment: Manual Logging with MLflow

### 📘 Multiple Choice (Choose the best answer)

**1. What is the purpose of `mlflow.log_param()` in MLflow?**    
A. To train a model with logged parameters    
B. To log runtime metrics during training    
C. To save the trained model to disk    
**D. To record a hyperparameter value used in an experiment** ✅    

---

**2. What does `mlflow.start_run()` do?**    
A. Runs the model in a separate thread    
B. Initializes the MLflow tracking server    
**C. Starts a new context to log experiment data** ✅    
D. Automatically saves the model to the registry    

---

**3. Which metric indicates how much the model's prediction error is, on average, in the same unit as the target?**    
A. MSE    
**B. RMSE** ✅    
C. R²    
D. Accuracy    

---

**4. Which MLflow method is used to store the model artifact after training?**    
A. `mlflow.save_model()`    
**B. `mlflow.sklearn.log_model()`** ✅    
C. `mlflow.track_model()`        
D. `mlflow.write_model()`    

---

### ✏️ Short Answer

**5. Explain the difference between MSE and RMSE. When might RMSE be preferred for interpretation?**    
*Your answer should mention that MSE is the average squared error, while RMSE is its square root, offering a more interpretable value in the target’s original units.*

---

**6. What happens if you don’t use `mlflow.start_run()` before logging parameters or metrics?**    
*Answer: MLflow will not track anything unless it's within a `start_run()` context. You might get an error or silent failure to log.*

---

### 🧪 Mini Project

**7. Task: Change the Ridge regression model to use `Lasso` instead.**    

* Log the same `alpha` values    
* Log `mse` and `rmse`    
* Log the model using `mlflow.sklearn.log_model()`v
* Compare and sort runs to find the best-performing one    

✅ *Bonus*: Can you add R² score (`r2_score`) as a new metric?    

